In [1]:
# import dependencies
import os
import spotipy
import spotipy.util as util
import pandas as pd
import csv
from pprint import pprint

Client_ID = "7655188a18144343b07e44965cc38310"

Client_Secret = "2bec9b47853e4ea3a419418a211ec45d"

In [2]:
top_2017 = pd.read_csv("resources/featuresdf.csv")
top_2018 = pd.read_csv("resources/top2018.csv")
top_2019 = pd.read_csv("resources/top50.csv", encoding='cp1252')

In [51]:
top_2019.head()
search_data = []
tracks = list(top_2019['Track.Name'])
artists = list(top_2019['Artist.Name'])


for i in range(len(tracks)):
    search_data.append([tracks[i],artists[i]])
    
#search_data

In [33]:
top_2017.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713.0,4.0
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.813,88.931,228827.0,4.0
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200.0,4.0
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.446,103.019,247160.0,4.0
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,-4.284,1.0,0.0367,0.0552,0.000000,0.1670,0.811,80.924,288600.0,4.0


In [55]:
#top_2017.head()
#https://open.spotify.com/user/1298997744?si=ijwp9NZOQoyjO9qOZliclw
SPOTIPY_CLIENT_ID='7655188a18144343b07e44965cc38310'
SPOTIPY_CLIENT_SECRET='2bec9b47853e4ea3a419418a211ec45d'
SPOTIPY_REDIRECT_URI='https://google.com/'
scope = 'user-library-read'
# Grab username
username = "1298997744"

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)


results = []
# Create our spotify object
for query in search_data:
    if token:
        track = query[0]
        sp = spotipy.Spotify(auth=token)
        result = sp.search(q=track, type='track')
        #pprint(result)
        results.append(result)
    else:
        print("Can't get token for", username)
    
pprint(results[0])
    
# song_ids = []

# if token:
#     sp = spotipy.Spotify(auth=token)
#     result = sp.search(q=name, type='track')
#     #pprint(result)
#     results.append(result)
# else:
#     print("Can't get token for", username)

# #util.prompt_for_user_token(username,scope,client_id='your-app-redirect-url',client_secret='your-app-redirect-url',redirect_uri='your-app-redirect-url')
# song_id = results[0]['tracks']['items'][0]['id']

# #sp.audio_analysis(song_id)
# sp.audio_features(song_id)

# sp.artist(artist_id="7n2wHs1TKAczGzO7Dd2rGr")
# #sp.track(song_id)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Se%C3%B1orita&type=track&offset=0&limit=10',
            'items': [{'album': {'album_type': 'single',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7n2wHs1TKAczGzO7Dd2rGr'},
                                              'href': 'https://api.spotify.com/v1/artists/7n2wHs1TKAczGzO7Dd2rGr',
                                              'id': '7n2wHs1TKAczGzO7Dd2rGr',
                                              'name': 'Shawn Mendes',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:7n2wHs1TKAczGzO7Dd2rGr'},
                                             {'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
                                              'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
                                

                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0GOx72r5AAEKRGQFn3xqXK'},
                                              'href': 'https://api.spotify.com/v1/artists/0GOx72r5AAEKRGQFn3xqXK',
                                              'id': '0GOx72r5AAEKRGQFn3xqXK',
                                              'name': 'Maître Gims',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:0GOx72r5AAEKRGQFn3xqXK'},
                                             {'external_urls': {'spotify': 'https://open.spotify.com/artist/1r4hJ1h58CWwUQe3MxPuau'},
                                              'href': 'https://api.spotify.com/v1/artists/1r4hJ1h58CWwUQe3MxPuau',
                                              'id': '1r4hJ1h58CWwUQe3MxPuau',
                                              'name': 'Maluma',
                                              'type': 